In [43]:
import Pkg
Pkg.update()

pkgs = [
    "Revise",
    "MetaGraphs",
    "Graphs",
    "JSON",
    "uCSV",
    "DataFrames",
    "Dates",
    "Primes",
    "Kmers",
    "BioSequences",
    "FASTX",
    "ProgressMeter",
    "Random"
]

Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

# Pkg.add(url="https://github.com/cjprybol/Mycelia.git", rev="master")
# Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

    Updating registry at `/opt/julia/registries/General`
  No Changes to `~/work/Mycelia/Project.toml`
  No Changes to `~/work/Mycelia/Manifest.toml`
Precompiling project...
  ✓ Mycelia
  1 dependency successfully precompiled in 20 seconds (232 already precompiled)
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version
   Resolving package versions...
  No Changes to `~/work/Mycelia/Project.toml`
  No Changes to `~/work/Mycelia/Manifest.toml`


In [2]:
#papermill parameters
base_directory = "$(homedir())/workspace/sars-cov2-pangenome-analysis/"
sequences_directory = "$(base_directory)/data/sequences"
metadata_file = "$(base_directory)/metadata/sequences.csv"
working_directory = "$(homedir())/workspace/scratch/$(Dates.today())"

"/home/jovyan/workspace/scratch/2022-09-18"

In [105]:
remote_neo4j_address = ENV["NEO4J_URL"]
remote_neo4j_password = ENV["NEO4J_PASSWORD"]
local_neo4j_bolt_address = "bolt://localhost:7687"
local_neo4j_http_address = "neo4j://localhost:7474"
local_neo4j_username = "neo4j"
# local_neo4j_password = "neo4j"
# local_neo4j_password = Random.randstring(7)
local_neo4j_password = "ii0sRIc"

"ii0sRIc"

In [117]:
# # update password - only need to do this on initial setup
cmd = "ALTER CURRENT USER SET PASSWORD FROM 'neo4j' TO '$(local_neo4j_password)'"
run(Mycelia.cypher(cmd, address=local_neo4j_bolt_address, password="neo4j", username="neo4j", database="system"))

Process(`/home/jovyan/.local/cypher-shell/cypher-shell --address bolt://localhost:7687 --username neo4j --password neo4j --format auto --database system "ALTER CURRENT USER SET PASSWORD FROM 'neo4j' TO 'ii0sRIc'"`, ProcessExited(0))

In [3]:
mkpath(working_directory)
cd(working_directory)

In [4]:
# f = metadata_file
f = open(`head $metadata_file`)
@time sequence_metadata = DataFrames.DataFrame(uCSV.read(f, quotes='"', header=1, typedetectrows=100)...)

  4.512879 seconds (8.11 M allocations: 492.936 MiB, 4.09% gc time, 77.36% compilation time)


,Accession,SRA_Accession,Submitters
,String,String,String
1,NC_045512.2,,"Wu,F., Zhao,S., Yu,B., Chen,Y.M., Wang,W., Song,Z.G., Hu,Y., Tao,Z.W., Tian,J.H., Pei,Y.Y., Yuan,M.L., Zhang,Y.L., Dai,F.H., Liu,Y., Wang,Q.M., Zheng,J.J., Xu,L., Holmes,E.C., Zhang,Y.Z., Baranov,P.V., Henderson,C.M., Anderson,C.B., Gesteland,R.F., Atkins,J.F., Howard,M.T., Robertson,M.P., Igel,H., Baertsch,R., Haussler,D., Ares,M. Jr., Scott,W.G., Williams,G.D., Chang,R.Y., Brian,D.A., Chen,Y.-M., Song,Z.-G., Tao,Z.-W., Tian,J.-H., Pei,Y.-Y., Zhang,Y.-L., Dai,F.-H., Wang,Q.-M., Zheng,J.-J., Zhang,Y.-Z."
2,OP164776.1,,"Afrad Moon,M.H., Qadri,F."
3,OP164777.1,,"Afrad Moon,M.H., Qadri,F."
4,OP164778.1,,"Afrad Moon,M.H., Qadri,F."
5,OP164779.1,,"Afrad Moon,M.H., Qadri,F."
6,OP164780.1,,"Afrad Moon,M.H., Qadri,F."
7,OP164781.1,,"Afrad Moon,M.H., Qadri,F."
8,OP164782.1,,"Afrad Moon,M.H., Qadri,F."
9,OP164783.1,,"Afrad Moon,M.H., Qadri,F."


In [5]:
fastx_files = map(x -> "$(sequences_directory)/$(x).fna", sequence_metadata[!, "Accession"])

9-element Vector{String}:
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/NC_045512.2.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164776.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164777.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164778.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164779.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164780.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164781.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164782.1.fna"
 "/home/jovyan/workspace/sars-cov2-pangenome-analysis//data/sequences/OP164783.1.fna"

In [6]:
graph = Mycelia.initialize_graph()
graph = Mycelia.add_fastx_to_graph!(graph, fastx_files)
Mycelia.add_metadata_from_table!(graph, sequence_metadata, identifier_column = "Accession")

{9, 0} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [8]:
# max_k = Mycelia.assess_dnamer_saturation(fastx_files)
# max_k = 31
# min_k=17
# max_k=17
min_k=max_k=31
kmer_sizes = Primes.primes(min_k, max_k)

1-element Vector{Int64}:
 31

In [9]:
for kmer_size in kmer_sizes
    @show kmer_size
    @time Mycelia.add_fasta_record_kmers_to_graph!(graph, kmer_size)
end

kmer_size = 31
  7.696067 seconds (21.09 M allocations: 1.614 GiB, 20.85% gc time)


In [11]:
for kmer_size in kmer_sizes
    @show kmer_size
    @time Mycelia.graph_to_gfa(graph, kmer_size)
end

kmer_size = 31
  0.810414 seconds (2.73 M allocations: 149.551 MiB)


In [12]:
for gfa in filter(x -> occursin(r"\.gfa$", x), readdir(working_directory, join=true))
    @show gfa
    for nodewidth in [100, 500, 1000]
        gfa_img = "$(gfa).$(nodewidth).jpg"
        if !isfile(gfa_img)
            run(`Bandage image $gfa $gfa_img --deppower 1 --depwidth 1 --nodewidth $(nodewidth)`)
        end
    end
end

gfa = "/home/jovyan/workspace/scratch/2022-09-18/31.gfa"


QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jovyan'
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jovyan'
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jovyan'


In [37]:
Mycelia.list_databases(address=remote_neo4j_address, username="neo4j", password=remote_neo4j_password)

,name,address,role,requestedStatus,currentStatus,error
,String,String,String,String,String,String
1,neo4j,bolt.lcfta.com:7687,standalone,online,online,
2,system,bolt.lcfta.com:7687,standalone,online,online,


In [57]:
function create_node_constraints(graph; address, username="neo4j", password, database="neo4j")
    node_types = unique(MetaGraphs.props(graph, v)[:TYPE] for v in Graphs.vertices(graph))
    node_type_strings = map(t -> Mycelia.type_to_string(t), node_types)
    for t in node_type_strings
        cmd = "CREATE CONSTRAINT ON (t:`$(t)`) ASSERT t.identifier IS UNIQUE"
        try
            cypher = Mycelia.cypher(cmd, address = address, username = username, password = password, database = database)
            @show cypher
            @time run(cypher)
        catch
            continue
        end
    end
end

create_node_constraints (generic function with 1 method)

In [122]:
create_node_constraints(graph, address=remote_neo4j_address, password=remote_neo4j_password)

cypher = `/home/jovyan/.local/cypher-shell/cypher-shell --address neo4j+s://bolt.lcfta.com:7687 --username neo4j --password G3nom3sRu13 --format auto --database neo4j 'CREATE CONSTRAINT ON (t:\`FASTX.FASTA.Record\`) ASSERT t.identifier IS UNIQUE'`
cypher = `/home/jovyan/.local/cypher-shell/cypher-shell --address neo4j+s://bolt.lcfta.com:7687 --username neo4j --password G3nom3sRu13 --format auto --database neo4j 'CREATE CONSTRAINT ON (t:\`Kmers.DNAKmer{31}\`) ASSERT t.identifier IS UNIQUE'`


An equivalent constraint already exists, 'Constraint( id=4, name='constraint_a8f8f2de', type='UNIQUENESS', schema=(:FASTX.FASTA.Record {identifier}), ownedIndex=3 )'.
An equivalent constraint already exists, 'Constraint( id=26, name='constraint_f6f7a729', type='UNIQUENESS', schema=(:Kmers.DNAKmer{31} {identifier}), ownedIndex=25 )'.


In [113]:
# run(`/home/jovyan/.local/neo4j-community-4.4.11/bin/neo4j stop`)
# run(`/home/jovyan/.local/neo4j-community-4.4.11/bin/neo4j start`)
# run(`/home/jovyan/.local/neo4j-community-4.4.11/bin/neo4j status`)

Neo4j is running at pid 88320


Process(`/home/jovyan/.local/neo4j-community-4.4.11/bin/neo4j status`, ProcessExited(0))

In [ ]:
#### This is all for uploading files

# database_id = "41a45c7d"
# DOMAIN = "41a45c7d.databases.neo4j.io"
# NODES_FILE = "$(DIR)/ncbi_taxonomy.nodes.tsv"
# EDGES_FILE = "$(DIR)/ncbi_taxonomy.edges.tsv"
# USERNAME="neo4j"
# PASSWORD=readline(joinpath(homedir(), ".config", "neo4j", "ncbi-taxonomy.password.txt"));
# ADDRESS="neo4j+s://$(database_id).databases.neo4j.io:7687"
# ADDRESS="neo4j+s://$(DOMAIN):7687"
# NEO4J_IMPORT_DIRECTORY="/var/lib/neo4j/import"
# DATABASE = "neo4j"
# run(`sudo neo4j start`)
# run(`neo4j status`)
# run(`sudo neo4j-admin set-initial-password testpass`)

# had to stop and restart to get the password change to take effect
# run(`sudo neo4j stop`)
# run(`sudo neo4j start`)

# # need to cleanup identifiers to make them valid in Neo4j
# node_table = joint_node_metadata
# for re in (r"\s+", r"-")
#     for col in filter(n -> occursin(re, n), names(node_table))
#         rename_map = col => replace(col, re => "_")
#         @show rename_map
#         DataFrames.rename!(node_table, rename_map)
#     end
# end

# # cleanup quotes
# ProgressMeter.@showprogress for col in names(node_table)
#     # strip away single-quotes
#     node_table[!, col] .= map(x -> x isa AbstractString ? replace(x, "'" => "") : x, node_table[!, col])
#     # and double quotes
#     node_table[!, col] .= map(x -> x isa AbstractString ? replace(x, '"' => "") : x, node_table[!, col])
# end

# # THIS IS SO IMPORTANT FOR MAKING THINGS GO FAST
# # AUTO SETS AN INDEX ON THESE PROPERTIES
# cmd = "CREATE CONSTRAINT ON (t:Taxonomy) ASSERT t.tax_id IS UNIQUE"
# @time run(cypher(address = local_address, username = USERNAME, password = local_password, database = DATABASE, cmd = cmd))

# cmd = "CREATE CONSTRAINT ON (t:Taxonomy) ASSERT t.`scientific name` IS UNIQUE"
# @time run(cypher(address = local_address, username = USERNAME, password = local_password, database = DATABASE, cmd = cmd))

# cmd = "CREATE CONSTRAINT ON (t:Taxonomy) ASSERT t.identifier IS UNIQUE"
# @time run(cypher(address = local_address, username = USERNAME, password = local_password, database = DATABASE, cmd = cmd))

# window_size = 10000
# V = DataFrames.nrow(node_table)
# windows = [i:min(i+window_size-1,V) for i in 1:window_size:V]

# parameters = ["$(n): row.$(n)" for n in filter(x -> x != "TYPE", names(node_table))]
# parameters = "{" * join(parameters, ", ") * "}"

# ProgressMeter.@showprogress for (i, w) in enumerate(windows)
#     df_sub = node_table[w, :]
#     f = "node$i.tsv"
#     local_f_path = "$(temp_upload_dir)/$(f)"
#     uCSV.write(local_f_path, df_sub, delim='\t')
#     run(`chmod 777 $(local_f_path)`)
#     f_url = "file:///$(local_f_path)"
#     cmd =
#     """
#     LOAD CSV WITH HEADERS FROM '$(f_url)' AS row FIELDTERMINATOR '\t'
#     CREATE (node:$(NODE_TYPE) $(parameters))
#     """
#     cmd = rstrip(replace(cmd, '\n' => ' '))
#     cypher_cmd = Mycelia.cypher(address = local_address, username = USERNAME, password = local_password, database = DATABASE, cmd = cmd)
#     run(cypher_cmd) 
# end

# add the edges to it

# src_type = dst_type = "Taxonomy"
# edge_type = "IS_PARENT_OF"
# # upload_edge_type_over_url(src_type, dst_type, edge_type, graph, ADDRESS, USERNAME, PASSWORD, DATABASE)

# window_size = 10000
# V = DataFrames.nrow(edge_table)
# windows = [i:min(i+window_size-1,V) for i in 1:window_size:V]

# ProgressMeter.@showprogress for (i, w) in enumerate(windows)
#     df_sub = edge_table[w, :]
#     f = "edge$i.tsv"
#     local_f_path = "$(temp_upload_dir)/$(f)"
#     uCSV.write(local_f_path, df_sub, delim='\t')
#     run(`chmod 777 $(local_f_path)`)
#     f_url = "file:///$(local_f_path)"
#     cmd = 
#     """
#     LOAD CSV WITH HEADERS FROM '$(f_url)' AS row FIELDTERMINATOR '\t'
#     MATCH (src:$(src_type) {identifier: row.src})
#     MATCH (dst:$(dst_type) {identifier: row.dst})
#     MERGE (src)-[p:$(edge_type)]->(dst)
#     """
#     cmd = rstrip(replace(cmd, '\n' => ' '))
#     cypher_cmd = Mycelia.cypher(address = local_address, username = USERNAME, password = local_password, database = DATABASE, cmd = cmd)
#     run(cypher_cmd) 
# end

# # run(`sudo touch /etc/neo4j/neo4j.conf`)
# run(`sudo neo4j stop`)

# remote database needs to be running
# needs to be big enough
# leave off port from address

# run(`neo4j-admin push-to-cloud --overwrite --verbose --bolt-uri=$(ADDRESS) --username=$(USERNAME) --password=$(PASSWORD)`)
# run(`sudo neo4j-admin push-to-cloud --overwrite --verbose --dump-to "$(DIR)/test.db.dump" --bolt-uri=$(a) --username=$(USERNAME) --password=$(PASSWORD)`)
# run(`sudo neo4j-admin push-to-cloud --overwrite --verbose --bolt-uri=$(a) --username=$(USERNAME) --password=$(PASSWORD)`)

# https://github.com/cjprybol/Mycelia/blob/master/docs/_src/4.Reference/neo4j-notes.ipynb

In [ ]:
# for t in node_type_strings
#     cmd = "CREATE CONSTRAINT ON (t:`$(t)`) ASSERT t.identifier IS UNIQUE"
#     @time run(Mycelia.cypher(cmd, address = ENV["NEO4J_URL"], username = "neo4j", password = ENV["NEO4J_PASSWORD"], database = "neo4j"))
# end

In [119]:
function upload_node_over_api(graph, v; ADDRESS, USERNAME="neo4j", PASSWORD, DATABASE="neo4j")
    node_type = MetaGraphs.props(graph, v)[:TYPE]
    node_identifier = MetaGraphs.props(graph, v)[:identifier]
    node_parameters = filter(x -> 
            !(x[1] in (:TYPE, :identifier)) && 
            !(ismissing(x[2]) || isempty(x[2])), 
        MetaGraphs.props(graph, v))
    params_string = join(["$(string(key)): \"$(string(value))\"" for (key, value) in node_parameters], ", ")
    node_type_string = Mycelia.type_to_string(node_type)
    node_identifier_string = string(node_identifier)
    cmd = 
    """
    MERGE (`$(node_identifier_string)`:`$(node_type_string)` {$(params_string)})
    """
    cmd = strip(cmd)
    cypher_cmd = Mycelia.cypher(cmd, address = ADDRESS, username = USERNAME, password = PASSWORD, database = DATABASE)
    run(cypher_cmd)
end

function upload_nodes_over_api(graph; ADDRESS, USERNAME="neo4j", PASSWORD, DATABASE="neo4j")
    ProgressMeter.@showprogress for v in Graphs.vertices(graph)
        upload_node_over_api(graph, v, ADDRESS=ADDRESS, USERNAME=USERNAME, PASSWORD=PASSWORD, DATABASE=DATABASE)
    end    
end

upload_nodes_over_api (generic function with 1 method)

In [ ]:
# Progress:   0%|▏                                        |  ETA: 8:13:39
# upload_nodes_over_api(graph, ADDRESS=local_neo4j_bolt_address, PASSWORD=local_neo4j_password)
# Progress:   0%|                                         |  ETA: 11:38:25
upload_nodes_over_api(graph, ADDRESS=remote_neo4j_address, PASSWORD=remote_neo4j_password)

Progress:   1%|▎                                        |  ETA: 11:35:43

In [ ]:
upload_edges_over_api(graph, ADDRESS, PASSWORD)

In [ ]:
Mycelia.upload_edge_type_over_url_from_graph(src_type, dst_type, edge_type, graph, ADDRESS, USERNAME, PASSWORD, DATABASE; window_size=100)

# ProgressMeter.@showprogress for (i, w) in enumerate(windows)
#     df_sub = edge_table[w, :]
#     f = "edge$i.tsv"
#     local_f_path = "$(temp_upload_dir)/$(f)"
#     uCSV.write(local_f_path, df_sub, delim='\t')
#     run(`chmod 777 $(local_f_path)`)
#     f_url = "file:///$(local_f_path)"
#     cmd = 
#     """
#     LOAD CSV WITH HEADERS FROM '$(f_url)' AS row FIELDTERMINATOR '\t'
#     MATCH (src:$(src_type) {identifier: row.src})
#     MATCH (dst:$(dst_type) {identifier: row.dst})
#     MERGE (src)-[p:$(edge_type)]->(dst)
#     """
#     cmd = rstrip(replace(cmd, '\n' => ' '))
#     cypher_cmd = Mycelia.cypher(address = local_address, username = USERNAME, password = local_password, database = DATABASE, cmd = cmd)
#     run(cypher_cmd) 
# end